In [4]:
import openai
from openai import OpenAI
import os
import json
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
client = OpenAI(
    api_key = os.getenv('RUNPOD_TOKEN'),
    base_url = os.getenv('RUNPOD_CHATBOT_URL'),
)
model_name = os.getenv("MODEL_NAME")

In [6]:
def get_chatbot_response(client,model_name,messages,temperature=0):
    input_messages = []
    for message in messages:
        input_messages.append({"role": message["role"], "content": message["content"]})

    response = client.chat.completions.create(
        model=model_name,
        messages=input_messages,
        temperature=temperature,
        top_p=0.8,
        max_tokens=2000,
    ).choices[0].message.content    
    
    return response

# Get LLM response

In [ ]:
messages = [{"role":"system","content":"What is the capital of Egypt?"}]

response = get_chatbot_response(client,model_name,messages)

'The capital of Egypt is Cairo.'

In [ ]:
response

# Prompt Engineering

## Structured Output

In [5]:
system_prompt = """ 

You are a helpful assistant that answers questions about capitals of countries.

Your output should be in a structured json format exactly like the on below. You are not allowed to write anything other than the json object.
[
{
    "country": the country that you will get the capital of
    "capital": the capital of the country stated
}
]

"""

messages = [{"role":"system","content":system_prompt}]
messages.append({"role":"user","content":"What is the capital of Egypt?"})
response = get_chatbot_response(client,model_name,messages)
print(response)

[
{
    "country": "Egypt",
    "capital": "Cairo"
}
]


In [6]:
type(response)

str

In [7]:
json_response = json.loads(response)
json_response

[{'country': 'Egypt', 'capital': 'Cairo'}]

In [9]:
type(json_response[0]),json_response[0]['capital']

(dict, 'Cairo')

## Input Structuring

In [10]:
user_input = """
Get me the capitals of the following countries:
```
1. Egypt
2. Italy
3. Germany
```
"""

messages = [{"role":"system","content":system_prompt}]
messages.append({"role":"user","content":user_input})   
response = get_chatbot_response(client,model_name,messages)
print(response)

[
{
    "country": "Egypt",
    "capital": "Cairo"
},
{
    "country": "Italy",
    "capital": "Rome"
},
{
    "country": "Germany",
    "capital": "Berlin"
}
]


In [11]:
json_response = json.loads(response)
json_response

[{'country': 'Egypt', 'capital': 'Cairo'},
 {'country': 'Italy', 'capital': 'Rome'},
 {'country': 'Germany', 'capital': 'Berlin'}]

## Give the model time to think (Chain of thought)

In [12]:
user_prompt = """
Caculate the result of this equation: 41+21

Your output should be in a structured json format exactly like the on below. You are not allowed to write anything other than the json object.
{
    result: The final number resulted from calculating the equation above
}
"""

messages = [{"role":"user","content":user_prompt}]
response = get_chatbot_response(client,model_name,messages)
print(response)

{
    "result": 62
}


In [1]:
259/2*8654+91072*33-12971

4113098.0

In [7]:
user_prompt = """
Caculate the result of this equation: 259/2*8654+91072*33-12971

Your output should be in a structured json format exactly like the on below. You are not allowed to write anything other than the json object.
{
    result: The final number resulted from calculating the equation above
}
"""

messages = [{"role":"user","content":user_prompt}]
response = get_chatbot_response(client,model_name,messages)
print(response)

{
    "result": 1434921119
}


In [8]:
user_prompt = """
Caculate the result of this equation: 259/2*8654+91072*33-12971

Your output should be in a structured json format exactly like the on below. You are not allowed to write anything other than the json object.
{
    steps: This is where you solve the equation bit by bit following the BEDMAS order of operations. You need to show your work and calculate each step leading to the final result. Feel free to write in free text.
    result: The final number resulted from calculating the equation above
}
"""

messages = [{"role":"user","content":user_prompt}]
response = get_chatbot_response(client,model_name,messages)
print(response)

{
    "steps": "First, we need to follow the BEDMAS order of operations. 
    1. Divide 259 by 2: 259 / 2 = 129.5
    2. Multiply 129.5 by 8654: 129.5 * 8654 = 1121011
    3. Multiply 91072 by 33: 91072 * 33 = 3005016
    4. Add 1121011 and 3005016: 1121011 + 3005016 = 4126027
    5. Subtract 12971 from 4126027: 4126027 - 12971 = 4113056",
    "result": 4113056
}


In [9]:
4113098.0 -4113056

42.0

## RAG - Retrival Augmented Generation

In [11]:
user_prompt = """
What's new in iphone 16
"""
messages = [{"role":'user',"content":user_prompt}]
response = get_chatbot_response(client,model_name,messages)
print(response)

Since Apple has not officially announced the iPhone 16, I'll provide you with some rumored features and speculations based on industry trends and leaks. Keep in mind that these are not confirmed and may change when the official announcement is made.

**Rumored Features:**

1. **Design:** The iPhone 16 is expected to have a similar design to the iPhone 15, with a stainless steel frame and a tougher glass back. However, some rumors suggest a new " pill-shaped" notch or a smaller notch.
2. **Display:** The iPhone 16 might feature a higher refresh rate (up to 120Hz) and a higher resolution (up to 6.7 inches) on the Pro models.
3. **Camera:** The iPhone 16 is expected to have improved camera capabilities, including a new periscope lens, a wider-angle lens, and enhanced low-light performance.
4. **Battery Life:** The iPhone 16 might have a larger battery, which could provide up to 2 days of battery life.
5. **Charging:** The iPhone 16 could support faster charging speeds (up to 30W) and wire

In [12]:
iphone_16 = """
The iPhone 16 introduces several exciting updates, making it one of Apple's most advanced smartphones to date. It features a larger 6.1-inch display for the base model and a 6.7-inch screen for the iPhone 16 Plus, with thinner bezels and a more durable Ceramic Shield. The iPhone 16 Pro and Pro Max boast even larger displays, measuring 6.3 and 6.9 inches respectively, offering the thinnest bezels seen on any Apple product so far.

Powered by the new A18 chip (A18 Pro for the Pro models), these phones deliver significant performance improvements, with enhanced neural engine capabilities, faster GPU for gaming, and machine learning tasks. The camera systems are also upgraded, with the base iPhone 16 sporting a dual-camera setup with a 48MP main sensor. The Pro models offer a 48MP Ultra Wide and 5x telephoto camera, enhanced by Apple’s "Camera Control" button for more flexible photography options.

Apple also introduced advanced audio features like "Audio Mix," which uses machine learning to separate background sounds from speech, allowing for more refined audio capture during video recording. Battery life has been extended, especially in the iPhone 16 Pro Max, which is claimed to have the longest-lasting battery of any iPhone 
9TO5MAC

APPLEMAGAZINE
.

Additionally, Apple has switched to USB-C for faster charging and data transfer, and the Pro models now support up to 2x faster video encoding. The starting prices remain consistent with previous generations, with the iPhone 16 starting at $799, while the Pro models start at $999
"""

In [13]:
user_prompt = f"""
{iphone_16}

What's new in iphone 16
"""

messages = [{"role":'user',"content":user_prompt}]
response = get_chatbot_response(client,model_name,messages)
print(response)

According to the text, the new features and updates in the iPhone 16 include:

1. **Larger displays**: 
   - Base model: 6.1-inch display
   - iPhone 16 Plus: 6.7-inch screen
   - iPhone 16 Pro: 6.3-inch display
   - iPhone 16 Pro Max: 6.9-inch display

2. **Thinner bezels**: The iPhone 16 Pro and Pro Max have the thinnest bezels seen on any Apple product so far.

3. **Durable Ceramic Shield**: The iPhone 16 features a more durable Ceramic Shield.

4. **New A18 chip (A18 Pro for Pro models)**: The new chip delivers significant performance improvements, including:
   - Enhanced neural engine capabilities
   - Faster GPU for gaming
   - Machine learning tasks

5. **Upgraded camera systems**:
   - Base iPhone 16: Dual-camera setup with a 48MP main sensor
   - Pro models: 48MP Ultra Wide and 5x telephoto camera
   - Enhanced by Apple’s "Camera Control" button for more flexible photography options

6. **Advanced audio features**: "Audio Mix" uses machine learning to separate background soun